In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('../data/combined_data_v6.csv')
data['headline'] = data['headline'].astype('U')
data['year'] = data['year'].astype(int)
data = data.drop_duplicates().reset_index(drop = True)
data.shape

(439412, 5)

In [3]:
data.sample(n = 10)

,source,headline,year,date,is_sarcastic
131565,ABC Australia,cam mccarthy afl trade talk between fremantle ...,2016,2016-09-20,0
141073,ABC Australia,iraqi forces find australians is flag in mosul,2016,2016-11-18,0
82365,Huffington Post,Inmates In Flint Forced To Consume And Use Con...,2016,2016-02-05,0
198663,ABC Australia,old boys urge anglican church to hand over sch...,2018,2018-02-09,0
213062,ABC Australia,father murdered son to avoid missing afl grand...,2018,2018-06-11,0
91820,ABC Australia,greyhound racing national rallies call for end...,2016,2016-02-07,0
298492,npr,Wash. Investigates More Possible Coronavirus C...,2020,2020-03-01,0
334091,guardian,Rhode Island governor sees off leftwing challe...,2018,2018-09-13,0
278294,npr,'How To Think Like An Anthropologist' — And Wh...,2018,2018-02-13,0
278432,npr,French Food Waste Law Changing How Grocery Sto...,2018,2018-02-24,0


### Preprocessing Steps
1. Standardize common abbreviations* (WIP)
    1. u.s. --> usa
1. Lowercase
1. Expand Contractions
1. Optional:
    1. Remove Source Specific Language
    1. Remove Profanity
1. Remove Special Characters


In [4]:
def replace_words(text, replace_dict):
    tokens = []
    for w in text.split():
        word = w
        for t in replace_dict.keys():
            if w == t:
                word = replace_dict[t]
        tokens.append(word)
    
    return " ".join(tokens)

In [5]:
data['clean'] = data['headline']

## Standardize Common Abbreviations
translate_dict = {
    "US": "USA",
    "U.S.": "USA",
    "u.s.": "USA",
    "u.s": "USA",
}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, translate_dict))

In [6]:
# Lowercase
data['clean'] = data['clean'].apply(lambda x: x.lower())

In [7]:
# Expand Contractions
contractions_dict = { "ain't": "are not","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who's": "who is",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}



data['clean'] = data['clean'].apply(lambda x: replace_words(x, contractions_dict))


In [8]:
data.loc[data['headline'].str.contains("'ll"), ['headline','clean']][:5]

,headline,clean
170,6 summer salads you'll actually crave,6 summer salads you will actually crave
190,20 suede pieces you'll want to wear all spring,20 suede pieces you will want to wear all spring
581,you'll want to get inked after seeing this tat...,you will want to get inked after seeing this t...
621,the hilarious hipster classifieds you'll (prob...,the hilarious hipster classifieds you will (pr...
721,"apple fritter season is here, and so are the r...","apple fritter season is here, and so are the r..."


In [9]:
### Remove bad text:

def remove_artifacts(s):
    return re.sub('xe[0-9]+x[0-9]+x[0-9]+', '',s)

data['clean'] = data['clean'].apply(lambda x: remove_artifacts(x))

In [10]:
data.loc[data['headline'].str.contains("xe"), ['headline','clean']][:5]

,headline,clean
24,drug-resistant bacteria often lurk in children...,drug-resistant bacteria often lurk in children...
220,arkansas executes first inmate in 12 years,arkansas executes first inmate in 12 years
356,several women accuse progressive media executi...,several women accuse progressive media executi...
620,do you have to pay income taxes on social secu...,do you have to pay income taxes on social secu...
845,"grab your wine boxes, because 'will & grace' i...","grab your wine boxes, because 'will & grace' i..."


In [11]:
### Remove possessive 's
data['clean'] = data['clean'].apply(lambda x: re.sub("\'s$", '', x))
data['clean'] = data['clean'].apply(lambda x: re.sub("\'s ", ' ', x))

In [12]:
data.loc[data['headline'].str.contains("'s"), ['headline','clean']][:5]

,headline,clean
3,advancing the world's women,advancing the world women
6,friday's morning email: inside trump's presser...,friday morning email: inside trump presser for...
10,"actually, cnn's jeffrey lord has been 'indefen...","actually, cnn jeffrey lord has been 'indefensi..."
12,bloomberg's program to build better cities jus...,bloomberg program to build better cities just ...
21,gillian jacobs on what it's like to kiss adam ...,gillian jacobs on what it like to kiss adam brody


In [13]:
## Remove Special Characters
data['clean'] = data['clean'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x))

In [14]:
## Trim Extra White Space
data['clean'] = data['clean'].apply(lambda x: re.sub('\s{2,}', ' ', x))
data['clean'] = data['clean'].apply(lambda x: x.strip())

In [15]:
guardian_length = data[data.source == 'guardian'].shape[0]

Huff = data[data.source == 'Huffington Post']
ABC = data[data.source == 'ABC Australia']

other_sources = data[(data.source != 'Huffington Post') & (data.source != 'ABC Australia')]

Huff = Huff.sample(n=guardian_length)
ABC = ABC.sample(n=guardian_length)

data_sources = [Huff, ABC, other_sources]
rebalanced = pd.concat(data_sources)

rebalanced = rebalanced.loc[~rebalanced['clean'].isnull()]
rebalanced = rebalanced.drop_duplicates()
rebalanced.reset_index(drop=True, inplace = True)

rebalanced.shape

(246667, 6)

In [16]:
rebalanced[['source', 'year', 'clean', 'is_sarcastic']].to_csv('../data/combined_clean_v10.csv', index = False, sep = "|")

### Profanity and Source Specific Information Removal

In [17]:
file = open('../references/profanity_wordlist.txt','r')
words = []
for line in file:
    stripped_line = line.strip()
    words.append(stripped_line)

fake_swears = ['anus','aryan','bdsm','bondage','breasts','crap','drunk','enlargement','erection','erotic',
              'erotism','facial','fat','gay','gays','gigolo','glans','god','hell', 'hemp','heroin','herpes',
              'hitler','hiv','hookah','hooker','hooter','hooters','hump','humped','humping','incest','jerk',
              'junkie','junky','kill','klan','kraut','LEN','lesbians','lesbos','loin','loins','lube','lust',
              'lusting','lusty','masochist','masturbate','masturbating','masturbation','menses','menstruate',
               'menstruation','meth','molest','moron','muff', 'murder','naked','napalm','nazi','nazism','nipple',
               'nipples','nude', 'nudes','opiate','opium','oral','orally','organ','ovary','ovum','ovums','pawn',
               'pcp','pedo','pedophile','pedophilia','pedophiliac','penetrate','penetration','perversion','peyote',
               'phallic','pimp', 'playboy','pollock','porn', 'pornography','pornos','pot','potty','prick','pricks',
               'prostitute','prude','punky','queer','quicky','racy','rape','raped','raper','raping','rapist',
               'raunch','revue','rum','rump','sadism','sadist','sandbar','scantilly','screw','screwed','screwing',
               'scum','seaman','seamen','seduce', 'sex','sexual','slave','sleaze','sleazy','slope','smut','smutty',
               'snatch','sniper','snuff','sodom', 'souse', 'steamy', 'stoned', 'strip', 'strip club', 'stripclub',
               'stroke','stupied','suck','sucked','sucking','tampon','tawdry','thrust', 'toke','tramp', 'trashy',
               'ugly','unwed','urinal','urine','uterus','uzi','valium','viagra','virgin','vixen','vodka','vomit',
               'voyeur','vulgar','weed','weirdo','wench','yury']

real_swears = list()
for word in words:
    if word not in fake_swears:
        real_swears.append(word)


def profanity_remove(text):
    if type(text)!='str':
        text = str(text)
    initial_words = text.split()
    new_words = list(filter(lambda w: w not in real_swears, initial_words))
    new_text = ' '.join(new_words)
    return new_text

data['clean'] = data['clean'].apply(lambda x:profanity_remove(x))

In [18]:
words = ['huffpollster', 'huffpost', 'gps guide', 'on the looney front', 'morning email','talktome', 
         'daily meditation', 'sunday roundup','planet america', 'talking pictures','sunday session','tasmania', 
        'perth', 'png','adelaide', 'wa', 'afl', 'queensland', 'nt','canberra','nsw', 'aboriginal', 'sa','hobart', 
         'qld','melbourne', 'brisbane', 'sydney', 'socceroos',  'nrl','bushfire', 'tasmanian', 'just game', 
        'rba', 'tas', 'matildas', 'qna', 'nz', 'nbn', 'sydneys', 'cairns','australians', 'newcastle',
         'tasmanias', 'asx', 'ndis', 'gold coast', 'aflw', 'townsville', 'adelaides', 'aussie',
         'news brief', 'in pictures', 'the guardian view', 'weatherwatch','track of the day', 'dear therapist', 
         'photos of the week', 'the family weekly','the books briefing', 'america by air', 'gaffe track',
         'todays news', 'poem of the day','trump time capsule', 'poem of the week', 'photo contest',
         'time capsule', 'education stories','big stories','onion', 'things to know','strongsideweakside', 
         'the onions', 'your horoscopes','things to watch', 'week in pictures', 'fox news poll', 'fox nation', 
        'week in politics', 'sunday politics','nprreads','troll watch']


mask = data['clean'].str.contains(r'\b(?:{})\b'.format('|'.join(words)))
data = data[~mask]

In [19]:
## Trim Extra White Space
data['clean'] = data['clean'].apply(lambda x: re.sub('\s{2,}', ' ', x))
data['clean'] = data['clean'].apply(lambda x: x.strip())

In [20]:
guardian_length = data[data.source == 'guardian'].shape[0]

Huff = data[data.source == 'Huffington Post']
ABC = data[data.source == 'ABC Australia']

other_sources = data[(data.source != 'Huffington Post') & (data.source != 'ABC Australia')]

Huff = Huff.sample(n=guardian_length)
ABC = ABC.sample(n=guardian_length)

data_sources = [Huff, ABC, other_sources]
rebalanced = pd.concat(data_sources)

rebalanced = rebalanced.loc[~rebalanced['clean'].isnull()]
rebalanced = rebalanced.drop_duplicates()
rebalanced.reset_index(drop=True, inplace = True)

rebalanced.shape

(241413, 6)

In [21]:
rebalanced[['source', 'year', 'clean', 'is_sarcastic']].to_csv('../data/combined_clean_v10_nonsource.csv', index = False, sep = "|")